# electricity_market_player

> Fill in a module description here

In [ ]:
#| default_exp electricity_market_player

In [ ]:
#| hide
from nbdev.showdoc import *

In [1]:
#| export
from sb3_contrib import MaskablePPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv

from electricity_market.electricity_market_env import ElectricityMarketEnv


/Users/ofer/Downloads/02360018_final_project/.venv/lib/python3.12/site-packages/gymnasium/envs/registration.py:642: UserWarning: WARN: Overriding environment ElectricityMarketEnv-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


In [ ]:
#| export
env_config = {
    "battery_capacity": 10_000_000,
    "init_state_of_charge": 4_000_000,
    "init_current_demand_of_electricity": 1000,
    "init_current_price": 100
}
env = ElectricityMarketEnv(env_config)

env.reset(seed=12123)

model = MaskablePPO("MlpPolicy", env, verbose=0)


model.learn(total_timesteps=10_000)
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=1000)

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")


In [ ]:
!python -m rl_zoo3.train --algo ppo --env ElectricityMarketEnv-v0 -n 50000 --optimize --n-trials 1000 --n-jobs 2 --gym-packages electricity_market

In [ ]:
import optuna
from stable_baselines3 import PPO
from electricity_market.electricity_market_env import ElectricityMarketEnv

def optimize_agent(trial):
    # Define hyperparameters to optimize
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True)
    n_steps = trial.suggest_int('n_steps', 16, 2048, log=True)
    batch_size = trial.suggest_int('batch_size', 16, 256, log=True)

    # Create environment
    env = DummyVecEnv([lambda: ElectricityMarketEnv()])

    # Initialize PPO with trial hyperparameters
    model = PPO("MlpPolicy", env, learning_rate=learning_rate, n_steps=n_steps, batch_size=batch_size, verbose=0)

    # Train the model
    model.learn(total_timesteps=int(1e3))

    # Evaluate the model
    total_reward = 0
    obs = env.reset()
    for _ in range(1000):
        action, _ = model.predict(obs)
        obs, reward, done, _ = env.step(action)
        total_reward += reward
        if done:
            break

    return total_reward

# Set up Optuna study
study = optuna.create_study(direction="maximize")
study.optimize(optimize_agent, n_trials=10)

# Print best trial
print("Best trial:", study.best_trial)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()